# CrewAI Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and CrewAI, you will need to install the following packages:

```shell
pip install crewai unitycatalog-crewai
```

In [1]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [ ]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.ai.crewai.toolkit import UCFunctionToolkit
from unitycatalog.client import ApiClient, Configuration

In [3]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define functions and register them to Unity Catalog

In this next section, we'll be defining two Python functions and creating them within Unity Catalog so that they can be retrieved and used as tools within our CrewAI [Crews](https://docs.crewai.com/concepts/crews). 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [4]:
def calculate_delta_v(exhaust_velocity: float, initial_mass: float, final_mass: float) -> float:
    """
    Calculates the maximum change in velocity (delta-v) of a rocket using the Ideal Rocket Equation (Tsiolkovsky Equation).

    The Ideal Rocket Equation is given by:
        delta_v = exhaust_velocity * ln(initial_mass / final_mass)

    This calculation is tailored for missions departing from Earth.

    Args:
        exhaust_velocity: The effective exhaust velocity of the rocket's engine in meters per second (m/s).
        initial_mass: The initial total mass of the rocket, including propellant, in kilograms (kg).
        final_mass: The final total mass of the rocket after propellant has been expended, in kilograms (kg).

    Returns:
        The maximum change in velocity (delta-v) achievable by the rocket in meters per second (m/s).
    """
    import math  # local imports are required for functions in Unity Catalog

    if exhaust_velocity <= 0:
        raise ValueError("Exhaust velocity must be positive.")
    if initial_mass <= final_mass:
        raise ValueError("Initial mass must be greater than final mass.")

    mass_ratio = initial_mass / final_mass
    delta_v = exhaust_velocity * math.log(mass_ratio)
    return delta_v


def earth_escape_velocity() -> float:
    """
    Calculates the escape velocity required to break free from Earth's gravitational pull.

    The escape velocity from Earth is calculated using the formula:
        escape_velocity = sqrt(2 * G * M / R)

    where:
        G is the universal gravitational constant,
        M is the mass of Earth,
        R is the radius of Earth.

    Constants:
        - Gravitational Constant (G): 6.67430e-11 m^3 kg^-1 s^-2
        - Mass of Earth (M): 5.972e24 kg
        - Radius of Earth (R): 6.371e6 meters

    Returns:
        The escape velocity from Earth in meters per second (m/s).
    """
    import math

    GRAVITATIONAL_CONSTANT = 6.67430e-11  # m^3 kg^-1 s^-2
    EARTH_MASS = 5.972e24  # kg
    EARTH_RADIUS = 6.371e6  # meters

    escape_vel = math.sqrt(2 * GRAVITATIONAL_CONSTANT * EARTH_MASS / EARTH_RADIUS)
    return escape_vel

In [5]:
client.create_python_function(func=calculate_delta_v, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(name='calculate_delta_v', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='exhaust_velocity', type_text='DOUBLE', type_json='{"name": "exhaust_velocity", "type": "double", "nullable": false, "metadata": {"comment": "The effective exhaust velocity of the rocket\\"s engine in meters per second (m/s)."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The effective exhaust velocity of the rocket"s engine in meters per second (m/s).'), FunctionParameterInfo(name='initial_mass', type_text='DOUBLE', type_json='{"name": "initial_mass", "type": "double", "nullable": false, "metadata": {"comment": "The initial total mass of the rocket, including propellant, in kilograms (kg)."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=Non

In [6]:
client.create_python_function(
    func=earth_escape_velocity, catalog=CATALOG, schema=SCHEMA, replace=True
)

FunctionInfo(name='earth_escape_velocity', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[]), data_type=<ColumnTypeName.DOUBLE: 'DOUBLE'>, full_data_type='DOUBLE', return_params=None, routine_body='EXTERNAL', routine_definition='import math\n\nGRAVITATIONAL_CONSTANT = 6.67430e-11  # m^3 kg^-1 s^-2\nEARTH_MASS = 5.972e24  # kg\nEARTH_RADIUS = 6.371e6  # meters\n\nescape_vel = math.sqrt(2 * GRAVITATIONAL_CONSTANT * EARTH_MASS / EARTH_RADIUS)\nreturn escape_vel', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', is_null_call=False, security_type='DEFINER', specific_name='earth_escape_velocity', comment="Calculates the escape velocity required to break free from Earth's gravitational pull. The escape velocity from Earth is calculated using the formula: escape_velocity = sqrt(2 * G * M / R) where: G is the universal gravitational constant, M is the mass of Earth, R is the radius of Earth. Const

## Create a Toolkit instance of the functions

Now that the functions have been created within Unity Catalog, we can use the `unitycatalog-crewai` package to create a toolkit instance that our Agent will 'understand' as valid tools to use within its APIs. 

In [7]:
escape_vel_func = f"{CATALOG}.{SCHEMA}.earth_escape_velocity"
delta_v_func = f"{CATALOG}.{SCHEMA}.calculate_delta_v"

toolkit = UCFunctionToolkit(function_names=[escape_vel_func, delta_v_func], client=client)

tools = toolkit.tools

In [8]:
tools

[UnityCatalogTool(name='AICatalog__AISchema__earth_escape_velocity', description="Tool Name: AICatalog__AISchema__earth_escape_velocity\nTool Arguments: {}\nTool Description: Calculates the escape velocity required to break free from Earth's gravitational pull. The escape velocity from Earth is calculated using the formula: escape_velocity = sqrt(2 * G * M / R) where: G is the universal gravitational constant, M is the mass of Earth, R is the radius of Earth. Constants: - Gravitational Constant (G): 6.67430e-11 m^3 kg^-1 s^-2 - Mass of Earth (M): 5.972e24 kg - Radius of Earth (R): 6.371e6 meters", args_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.AICatalog__AISchema__earth_escape_velocity__params'>, description_updated=False, cache_function=<function UCFunctionToolkit.<lambda> at 0x306f67b00>, result_as_answer=False, fn=<function UCFunctionToolkit.uc_function_to_crewai_tool.<locals>.func at 0x10886dc60>, client_config={'uc': <unitycatalog.client.api.functions_api

## Create a Crew

Now that everything is all set for our tools that we want to use in our Crew, we can create it. 
We're going to define the Agents that will be in use, marking one of the tasks as requiring human input (so that we can ask a question to the Aeronautical Engineer). The tools will be added to this agent so that it can have access to our deterministic functions that reside within Unity Catalog. 

> **IMPORTANT DISCLAIMER**:
This implementation constitutes an extremely rudimentary approximation intended exclusively for educational and recreational purposes within the context of this demonstration. In real-world aerospace engineering and mission planning, rocket launch simulations necessitate the utilization of advanced numerical integration methodologies such as Runge-Kutta algorithms, comprehensive atmospheric modeling, intricate multi-stage rocket dynamic differential equations, and the incorporation of orbital mechanics solutions, including but not limited to the two-body problem for precise orbital trajectory calculations.

> **WARNING**:
The algorithms and models presented herein lack the sophistication, accuracy, and reliability required for any form of practical application, including but not limited to spacecraft navigation, mission-critical operations, or aerospace engineering endeavors.

> **CAUTION**:
Attempting to utilize this example for actual rocket launches or space travel may result in catastrophic failure and is unequivocally advised against. Always consult with qualified aerospace professionals and employ industry-standard simulation tools for any endeavors related to space exploration.

In [9]:
from crewai import Agent, Crew, Task

# Create agents
aeronautical_engineer = Agent(
    role="Aeronautical Engineer",
    goal="Answer questions regarding fundamental concepts in Physics as they relate to Aeronautical Engineering",
    backstory="Works at NASA. Loves Physics and Math. Thoroughly believes that The Expanse is superior to Star Wars.",
    expected_output="Either the result of a mathematical calculation or a result and an explanation of the calculation",
    tools=tools,
    verbose=True,
)

mission_specialist = Agent(
    role="Auditor",
    goal="Ensures that the Aeronautical Engineer provides correct answers using deterministic tools",
    backstory="Loves space. Likes boring days at work. Likes seeing flags planted on planetary bodies.",
    expected_output="A full explanation of the calculated result with both a lay person's and a highly technical audience in mind",
    verbose=True,
)

# Define tasks
research = Task(
    description="Call tools that are relevant to the posed question",
    expected_output="string",
    agent=aeronautical_engineer,
    human_input=True,
)

review = Task(
    description="Review and evaluate the tool call output and provide a final answer to the question",
    expected_output="string",
    agent=mission_specialist,
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[aeronautical_engineer, mission_specialist],
    tasks=[research, review],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

# Human input (try something different and see what happens!):
# Can I get to low Earth orbit with my 4500000kg rocket that is holding 4450000kg of fuel that has engine exhaust velocity of 19800m/s?

 
[2024-11-26 15:54:05][INFO]: Planning the crew execution
# Agent: Aeronautical Engineer
## Task: Call tools that are relevant to the posed questionBased on the review of outputs, collate findings and direct responses to the original query while ensuring clarity and accuracy. Highlight any specific aeronautical engineering principles that are relevant to the results.


# Agent: Aeronautical Engineer
## Thought: I need to determine whether I should calculate the escape velocity from Earth or find the delta-v of a rocket. It depends on the context of the question posed. If the goal is to understand the conditions required for spacecraft to leave Earth's gravitational influence, then escape velocity is relevant. If the question relates to the performance of a rocket in terms of changing its velocity to reach a specific orbit or destination, then the delta-v is more pertinent. 
Since I'm currently equipped to provide calculations for both aspects, I will calculate the escape velocity from

 Can I get to low Earth orbit with my 4500000kg rocket that is holding 4450000kg of fuel that has engine exhaust velocity of 19800m/s?




# Agent: Aeronautical Engineer
## Thought: Thought: To determine if the rocket can reach low Earth orbit (LEO), I need to calculate the delta-v using the ideal rocket equation (Tsiolkovsky Equation). For this, I will use the mass of the rocket, the mass of the fuel, and the exhaust velocity of the rocket's engine.
The initial mass of the rocket includes the rocket itself plus the fuel, and the final mass is just the mass of the rocket after the fuel has been expended. 
- Initial mass: 4,500,000 kg (rocket + fuel)
- Final mass: 500,000 kg (rocket after fuel is spent)
- Exhaust velocity: 19,800 m/s
Now I will perform the delta-v calculation.
## Using tool: AICatalog__AISchema__calculate_delta_v
## Tool Input: 
"{\"initial_mass\": 4500000, \"final_mass\": 500000, \"exhaust_velocity\": 19800}"
## Tool Output: 
{"format": "SCALAR", "value": "43505.046631257144"}


# Agent: Aeronautical Engineer
## Final Answer: 
The delta-v calculated for your rocket, which has an initial mass of 4,500,00

CrewOutput(raw="### Structured Evaluation Log\n\n**1. Context of Evaluation**  \n- The rocket involved has an initial mass of 4,500,000 kg, inclusive of 4,450,000 kg of fuel.  \n- The engine exhaust velocity is established at 19,800 m/s.\n\n**2. Delta-V Calculation**  \n- The calculated delta-v for the rocket is approximately 43,505 m/s.  \n\n**3. Requirement for Low Earth Orbit (LEO)**  \n- The typical delta-v required to achieve LEO is around 7,800 m/s.\n\n**4. Evaluation**  \n- Comparison of calculated delta-v to LEO delta-v:  \n  - **Calculated Delta-V:** 43,505 m/s  \n  - **Required for LEO:** 7,800 m/s  \n  - **Conclusion:** The calculated delta-v exceeds the required delta-v for LEO, confirming that the rocket can successfully reach this orbit.\n\n**5. Fundamental Principle**  \n- This evaluation reinforces a key principle in aeronautical engineering: the efficiency of rocket propulsion systems significantly influences the performance of launch vehicles and their ability to atta